In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from vae_injected import VAEInjected

In [2]:
## Loading dataset
model_name = "inception"
assert model_name in ['inception', 'vgg', 'resnet', 'mobilenet']
labels = np.load('./data/LFSD_labels.npy')
depths = np.load('./data/LFSD_depths_repeated.npy')
imgs = np.load('./data/LFSD_imgs.npy')
masks = np.load('./data/LFSD_masks_single.npy')
depths_feat = np.load('./data/LFSD_depths_repeated_%s_feat.npy' % model_name)
imgs_feat = np.load('./data/LFSD_imgs_%s_feat.npy' % model_name)

def built_batch_iterator(num_data, batch_size):
    idx = np.random.permutation(num_data)
    return [idx[i:i + batch_size] for i in range(0, num_data, batch_size)]

batch_idxs = built_batch_iterator(len(labels), 64)
dataset = []
for idx in batch_idxs:
    img_batch, depth_batch, mask_batch = imgs[idx], depths[idx], masks[idx]
    img_feat_batch, depth_feat_batch = imgs_feat[idx], depths_feat[idx]
    dataset.append((img_batch, img_feat_batch, depth_batch, depth_feat_batch, mask_batch))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.3)
print(len(train_dataset), len(test_dataset))

9 5


In [3]:
latent_dim = 2048
vae = VAEInjected(latent_dim)
learning_rate = 1e-4
vae.compile(optimizer=Adam(learning_rate))
epochs = 10

def merge_images(image_batch, size):
    h,w = image_batch.shape[1], image_batch.shape[2]
    img = np.zeros((int(h*size[0]), w*size[1]))
    for idx, im in enumerate(image_batch):
        im = np.squeeze(im, axis=2)
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = im
    return img

losses_across_epochs = {
    "loss": [],
    "reconstruction_loss": [],
    "kl_loss": [],
}
batch_num = len(train_dataset)
for i in range(epochs):
    print("Epoch %d: " % i)
    for k, v in losses_across_epochs.items():
        losses_across_epochs[k].append(0)
    for data in train_dataset:
        cur_loss = vae.train_step(data)
        for k, v in cur_loss.items():
            losses_across_epochs[k][-1] += cur_loss[k].numpy() / batch_num
        generated_image = vae.sample(data)
    print("Total loss: %d" % losses_across_epochs['loss'][-1])
    im_merged = merge_images(generated_image.numpy(), [8,8])
    plt.imsave('./images/vae_injected/%d.png' % i, im_merged, cmap='gray')
for k, v in losses_across_epochs.items():
    np.save('./results/vae_injected/%s' % k, np.array(v))

Epoch 0: 
Total loss: 2992122
Epoch 1: 
Total loss: 2790143


KeyboardInterrupt: 

In [4]:
test_loss = 0
for i, data in enumerate(test_dataset):
    _, _, _, _, mask_batch = data
    generated_image = vae.sample(data)
    reconstruction_loss = tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(mask_batch, generated_image), [1,2]
    )
    test_loss += tf.reduce_mean(reconstruction_loss).numpy()
    im_merged = merge_images(generated_image.numpy(), [8,8])
    plt.imsave('./images/vae_injected/test_batch_%d.png' % i, im_merged, cmap='gray')
    
test_loss = test_loss / len(test_dataset)
np.save('./results/vae_injected/test_loss', np.array([test_loss]))

In [13]:
from skimage.transform import resize
from tensorflow.keras import datasets

train_dataset, test_dataset = None, None

(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()
result = []
train_feats = np.load('./data/CIFAR100_%s_train_feat.npy' % model_name)
test_feats = np.load('./data/CIFAR100_%s_test_feat.npy' % model_name)

for img, img_feat in tqdm(zip(train_images, train_feats)):
    img = np.expand_dims(resize(img, (256, 256, 3)), 0)
    img_feat = np.expand_dims(img_feat, 0)
    activation, _, _ = vae.encode((img, img_feat, None, None, None), rand_depth=True)
    activation = np.squeeze(activation, axis=0)
    result.append(activation)

np.save('./data/CIFAR100_vae_injected_%s_encoding_train.npy' % model_name, np.array(result))

0it [00:00, ?it/s](2048,)



In [ ]:
for img, img_feat in tqdm(zip(test_images, test_feats)):
    img = np.expand_dims(resize(img, (256, 256, 3)), 0)
    img_feat = np.expand_dims(img_feat, 0)
    activation, _, _ = vae.encode((img, img_feat, None, None, None), rand_depth=True)
    activation = np.squeeze(activation, axis=0)
    result.append(activation)

np.save('./data/CIFAR100_vae_injected_%s_encoding_test.npy' % model_name, np.array(result))